In [2]:
from app.core.config import settings
from app.database.mysql import get_mysql_connection
connection = get_mysql_connection()

2024-12-22 02:00:09,775 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_URI
2024-12-22 02:00:09,955 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_USER
2024-12-22 02:00:10,140 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_PASSWORD
2024-12-22 02:00:10,328 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_DB_NAME
2024-12-22 02:00:10,512 - fastapi_project - INFO - Retrieved parameter: /tailorlink/openai/OPENAI_API_KEY
2024-12-22 02:00:10,706 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_URI
2024-12-22 02:00:10,890 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_TOKEN
2024-12-22 02:00:11,072 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_API_KEY
2024-12-22 02:00:11,266 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_ENDPOINT
2024-12-22 02:00:11,413 - fastapi_project - INFO - Co

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
        """
        당신은 제네시스 차량 매뉴얼에 대한 전문 Q&A 도우미입니다.
        사용자가 질문을 하면 다음 규칙을 따르세요:

        먼저, 사용자가 한 질문이 제네시스 차량과 관련 있는지 판단하세요.

        만약 질문 내용이 제네시스 차량(g90,gv90,g80,gv80,g70,gv70 등등 ) 관련이 아닌 경우(예: 날씨, 음식, 다른 브랜드 차량 등), "제네시스 차량 관련된 질문만 가능합니다." 라고 한 줄로 출력하세요.
        제네시스 차량 관련 질문이더라도, 다른 브랜드(예: 기아, 현대, 벤츠 등)를 언급하거나 비교하는 부분이 있다면 해당 질문은 무효로 간주하고, "제네시스 차량 관련된 질문만 가능합니다." 라고 한 줄로 출력하세요.
        질문이 일반적인 차량 사용에 관련된 질문 이라면 제네시스 판단하세요.
        질문이 순수하게 제네시스 차량과 관련된 경우, 해당 질문이 단순한 하나의 질문인지, 아니면 여러 질문이 복합적으로 포함된 복합질문인지 판단하세요.

        복합질문이라면, 각 서브질문을 명확히 나눈 뒤, 리스트 형태로 출력하세요. (예: ["이 차량은 하이브리드인가요?", "엔진 오일은 어디서 교환하나요?"])
        출력은 조건에 따라 다음 두 가지 형식 중 하나를 따릅니다.

        비관련 혹은 타사 브랜드 언급 시:
        "제네시스 차량 관련된 질문만 가능합니다." (한 줄)
        제네시스 관련 복합질문 시:
        ["첫 번째 하위 질문", "두 번째 하위 질문", ...]
        예를 들어: ["이 차량은 하이브리드인가요?", "엔진 오일은 어디서 교환하나요?"]

        # 입력 질문: {question}
        # Output Format: {format_instructions}
        """
    )
from app.features.manual.models.model import create_openai_model
llm = create_openai_model('gpt-4o-mini')

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from app.features.manual.models.schemas import QuestionList

parser=JsonOutputParser(pydantic_object=QuestionList)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

res = chain.invoke({'question':'g90 시동거는 법 알려줘'})
print(res)
res1 =chain.invoke({'question':'티비시청?'})
print(res1)
res2 =chain.invoke({'question':'차량 시동이 안켜지는데 에어컨 때문이야?'})
print(res2)

{'vailid_question': True, 'print': '', 'question_list': ['g90 시동 거는 법 알려줘']}
{'vailid_question': False, 'print': '제네시스 차량 관련된 질문만 가능합니다.', 'question_list': []}
{'vailid_question': True, 'print': '', 'question_list': ['차량 시동이 안켜지는 이유는 무엇인가요?', '에어컨이 시동에 영향을 미치나요?']}


In [9]:
res3 =chain.invoke({'question':'g90 눈 올때 어떻게 하면돼? 그리고 차량 스펙 알려줘'})
print(res3)

{'vailid_question': True, 'print': '', 'question_list': ['눈 올 때 차량은 어떻게 운전해야 하나요?', '차량 스펙은 무엇인가요?']}


# DB 조회

In [6]:
from app.database.milvus import get_milvus_client
get_milvus_client()
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
        """
        사용자의 질문에 대해 차량 매뉴얼 벡터 데이터베이스를 조회하여 답변을 제공하십시오. 답변에는 사용된 참조 문서의 출처를 명확히 적어주세요.

        # Steps

        1. 사용자의 질문을 명확히 이해하고 필요한 정보를 식별합니다.
        2. 차량 매뉴얼의 벡터 데이터베이스를 조회하여 관련 정보를 검색합니다.
        3. 찾은 정보를 바탕으로 사용자 질문에 대한 정확하고 간결한 답변을 작성합니다.
        4. 답변에 사용한 참조 문서의 출처를 꼭 기재합니다.

        # Output Format

        - 사용자 질문에 대한 답변: [응답 내용]
        - 출처: [참조 문서 제목 및 페이지 또는 섹션 번호]

        # Examples

        **입력 예시:**
        "에어필터 교체 방법을 알고 싶어요."

        **출력 예시:**
        #에어필터 교체 방법 
          에어필터는 보통 차량 엔진룸에 위치해 있으며, 커버를 열고 고정 클립을 풀러 교체할 수 있습니다.
          출처: 차량 매뉴얼 5페이지

        # Notes

        - 가능한 한 정확하고 간결하게 정보를 제공하세요.
        - 찾은 정보가 명확하지 않을 경우, 사용자가 이해할 수 있도록 추가 설명을 포함할 수 있습니다.
        - 모든 답변에는 항상 출처를 포함해야 합니다.
        - 답변은 보기 좋게 마크다운으로 출력합니다.

        # 입력 질문: {question}
        """
    )
from app.features.manual.models.model import create_openai_model
llm = create_openai_model('gpt-4o-mini')
from app.features.manual.tools.tools import search_milvus

# query=['차량 시동이 안켜지는 이유는 무엇인가요?', '에어컨이 시동에 영향을 미치나요?']
query=['눈 올 때 차량은 어떻게 운전해야 하나요?', '차량 스펙은 무엇인가요?']
context = search_milvus.invoke({"query_list": query})

# from app.features.manual.models.reranker import bge_rf
# results=[]
# for index, query in enumerate(query):
#     result = bge_rf(
#         query=query[index],
#         documents=temp[index],
#         top_k=3,
#     )
#     results.append(result)
# context = results


2024-12-21 23:19:35,213 - fastapi_project - INFO - Connected to Milvus at https://in01-8b96d805fab7a0f.aws-ap-northeast-1.vectordb.zillizcloud.com:19533
E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [8]:
query= 'g90 눈 올때 어떻게 하면돼? 그리고 차량 스펙 알려줘'
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"context": context, "question":query})

print(response)

# 눈 올 때 차량 관리 방법
눈이 올 때는 차량의 안전을 위해 몇 가지 조치를 취하는 것이 중요합니다. 먼저, 타이어의 공기압을 확인하고 겨울용 타이어로 교체하는 것이 좋습니다. 또한, 와이퍼를 점검하고, 윈드실드에 얼음이 생기지 않도록 미리 제상제를 사용하는 것이 도움이 됩니다. 주행 시에는 속도를 줄이고, 충분한 안전거리를 유지하는 것이 중요합니다.

# 차량 스펙
G90의 기본 스펙은 다음과 같습니다:
- 엔진: 3.5L V6 또는 5.0L V8 옵션
- 출력: 380~420 마력
- 구동 방식: 후륜구동 또는 사륜구동
- 연비: 약 8~12 km/l (모델에 따라 다름)

출처: G90 차량 매뉴얼 12페이지 및 45페이지


# 할루시네이션 평가

In [9]:
docs = context
generation = response

In [10]:
from app.features.manual.models.prompt_templates import create_hallucination_prompt
from app.features.manual.models.model import create_openai_model
from app.features.manual.models.schemas import GradeHallucinations

hallucination_prompt = create_hallucination_prompt()
llm = create_openai_model('gpt-4o-mini')
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

hallucination_grader = hallucination_prompt | structured_llm_grader
res = hallucination_grader.invoke({"documents": docs, "generation": generation})

In [11]:
res.binary_score

'no'

# 이전 대화 내용

In [25]:
from app.features.manual.models.schemas import AnswerWithHistory
from langchain_core.prompts import PromptTemplate
from app.features.manual.models.model import create_openai_model

prompt = PromptTemplate.from_template(
    """
    다음은 이전 대화 내용만을 참고하여 응답을 생성하기 위한 프로세스입니다. 외부 지식이나 학습된 데이터를 사용하지 않고 오직 제공된 대화 내용을 기반으로 응답합니다.
    
    # 응답 가능성 판단 및 생성
    
    ## 절차
    1. **이전 대화 분석**: 이전 대화의 내용을 검토하여 대화의 주제 및 구체적으로 제기된 질문을 파악합니다.
    2. **질문 또는 프롬프트 식별**: 명시적 또는 암시적인 질문이나 요청을 식별합니다.
    3. **응답 가능성 평가**: 제공된 이전 대화 내용만으로 충분한 정보가 있는지 판단하고, 응답 가능 여부를 결정합니다.
       - 응답 가능하면 `'yes'`를 출력합니다.
       - 응답이 불가능하면 `'no'`를 출력합니다.
    4. **응답 생성**: 
       - 가능(`'yes'`)하면 이전 대화 내용만을 바탕으로 논리적이고 맥락에 맞는 응답을 작성합니다.
       - 불가능(`'no'`)하면 추가 정보가 필요하다는 점을 명시합니다.
    
    ## 제한 사항
    - 외부 지식, 학습된 데이터, 일반 상식 등을 사용하지 않습니다.
    - 이전 대화 내용 외의 정보를 가정하거나 추가하지 않습니다.
    
    ## 출력 형식
    - 출력은 **마크다운(Markdown)** 형식으로 작성합니다.
    - 응답 가능 여부(`yes` 또는 `no`)를 표시하고, 이에 따른 적절한 내용을 제공합니다.
    
    ## 이전 대화 내용: {chat_history}
    
    ## 질문: {question}
    """
)
llm = create_openai_model('gpt-4o-mini')
structured_llm_grader = llm.with_structured_output(AnswerWithHistory)
contextual_responder = prompt | structured_llm_grader

In [26]:
list = ['human: g90 눈 올때 어떻게 하면돼? 그리고 차량 스펙 알려줘', f'ai:{response}']

In [31]:
res = contextual_responder.invoke({"chat_history":list, "question":"비올떄 대처법?"})


In [32]:
print(res)

binary_score='no' answer='비 올 때 대처법에 대한 정보는 이전 대화 내용에 포함되어 있지 않습니다. 추가 정보가 필요합니다.'


In [1]:
from app.database.milvus import get_milvus_client
get_milvus_client()

2024-12-22 11:39:56,571 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_URI
2024-12-22 11:39:56,749 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_USER
2024-12-22 11:39:56,930 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_PASSWORD
2024-12-22 11:39:57,118 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_DB_NAME
2024-12-22 11:39:57,307 - fastapi_project - INFO - Retrieved parameter: /tailorlink/openai/OPENAI_API_KEY
2024-12-22 11:39:57,482 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_URI
2024-12-22 11:39:57,673 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_TOKEN
2024-12-22 11:39:57,867 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_API_KEY
2024-12-22 11:39:58,037 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_ENDPOINT
2024-12-22 11:39:58,357 - fastapi_project - INFO - Co

In [2]:
from app.features.manual.tools.tools import search_milvus
results = search_milvus.invoke({"query_list": ['시동거는법?'], "car_id":2})

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
results